In [1]:
%run basics
%matplotlib
import pysolar
import pytz

Using matplotlib backend: Qt4Agg


In [ ]:
def solar_altitude_IM(ldt,latitude,longitude,time_zone):
    # get a pytz object
    tz = pytz.timezone(time_zone)
    # use the same "current" date as Ian
    cdt = datetime.datetime(2013,6,1,0,0,0)
    # get the offset to UTC time as a timedelta object
    utcoffset = tz.utcoffset(cdt)
    # get the offset as hours
    GMT_zone = utcoffset.seconds/float(3600)
    # get the day of the year from local time
    DOY = numpy.array([dt.timetuple().tm_yday for dt in ldt])
    # For each day calculate equation of time correction, solar noon and declination
    eqn_of_time = 0.17*numpy.sin(4*numpy.pi*(DOY-80)/373)-0.129*numpy.sin(2*numpy.pi*(DOY-8)/355) # DiLaura (1984)
    solar_noon = 12+(GMT_zone*15.0-longitude)/360*24-eqn_of_time # Me
    decln = numpy.radians(23.4)*numpy.sin((DOY+284)/365.0*2*numpy.pi) # Oke (1987)
    # Create an hour angle array for each minute of day and each day of year
    hour_angle = numpy.tile(numpy.linspace(0,1439.0/1440*24,num=1440),(len(DOY),1))
    hour_angle = abs(numpy.radians((solar_noon.reshape(len(DOY),1)-hour_angle)*15))
    # Duplicate declination array for each time of day
    decln = numpy.tile(decln,(1440,1)).T
    # Calculate zenith angles
    array_z=numpy.arccos(numpy.sin(numpy.radians(lat_decdeg))*numpy.sin(array_decl)+numpy.cos(numpy.radians(lat_decdeg))*numpy.cos(array_decl)*numpy.cos(array_h))


In [2]:
site_name = "Whroo"
nc_name = "../Sites/Whroo/Data/Portal/Whroo_2014_L3.nc"
site_timezone = "Australia/Melbourne"
site_latitude = -36.67305
site_longitude = 145.02621

In [3]:
ds = qcio.nc_read_series(nc_name)
ldt = ds.series["DateTime"]["Data"]

In [7]:
tz = pytz.timezone(site_timezone)
cdt = datetime.datetime(2013,6,1,0,0,0)
utcoffset = tz.utcoffset(cdt)
GMT_zone = utcoffset.seconds/float(3600)
DOY = numpy.array([dt.timetuple().tm_yday for dt in ldt])
eqn_of_time = 0.17*numpy.sin(4*numpy.pi*(DOY-80)/373)-0.129*numpy.sin(2*numpy.pi*(DOY-8)/355)
solar_noon = 12+(GMT_zone*15.0-site_longitude)/360*24-eqn_of_time
decln = numpy.radians(23.4)*numpy.sin((DOY+284)/365.0*2*numpy.pi)
Hdh = numpy.array([dt.hour+dt.minute/float(60) for dt in ldt])
solar_zenith = numpy.arccos(numpy.sin(numpy.radians(site_latitude))
                            *numpy.sin(decln)
                            +numpy.cos(numpy.radians(site_latitude))
                            *numpy.cos(decln)*numpy.cos(Hdh))

In [9]:
fig=plt.figure()
plt.plot(ldt,solar_zenith)
plt.show()